# Tutotial DataReader

Utilisation des objets DataReader pour la lectures de données sources spécifiques. 
En résumé, que font les DataReader:
- Prise en charge d'un input specifique (une source de donnée comme par expl otoriver ou sensea).
- Prétraitmements éventuels sur la donnée brut (expl: passer sur un pas de temps horaire).
- Construction d'un dataset en sortie

In [ ]:
import os
cwd = os.getcwd()
new_cwd = os.path.join(cwd.rsplit("SeinAcoustic_DataAnalysis")[0], "SeinAcoustic_DataAnalysis") if not cwd.endswith("SeinAcoustic_DataAnalysis") else None
if new_cwd != None:
    os.chdir(new_cwd)

## 1. Descripteurs Acoustiques d'Otoriver : Otoriver DataReader 

Permettre la lecture et le prétraitement des données otoriver

In [ ]:
import pandas as pd
import plotly.express as px
import glob

In [ ]:
from DBBuilder.datareader import Otoriver_DataReader

# Initialisation de du DataReader
datareader = Otoriver_DataReader(timefreq="h") # Choisire le pas de temps de sortie
datareader.metadata # obtenir les métadonnées disponnible sur otoriver dans l'outil DataReader

In [ ]:
# Données brut otoriver
filepath = "data/otosea_EB/Export otosea - SIAAP-Bougival - 48943556.xlsx" 
df_otoriver_input = pd.read_excel(filepath) 
print("Données brut pour otoriver")
df_otoriver_input

In [ ]:
df_otoriver_output = datareader.from_xslx(filepath=filepath) # Lecture du fichier à l'aide du dataReader
print("Fichier de sortie pour otoriver")
df_otoriver_output

In [ ]:
fig_otoriver = px.line(df_otoriver_output, x="date", y=[c for c in df_otoriver_output.columns if c != "date"])
fig_otoriver.show()

## 2. Descripteurs Acoustiques de Sensea : Sensea DataReader

Permettre la lecture et le prétraitement des données sensea

In [ ]:
from DBBuilder.datareader import Sensea_DataReader

# Initialisation de du DataReader
datareader = Sensea_DataReader(timefreq="h") # Choisire le pas de temps de sortie
datareader.metadata # obtenir les métadonnées disponnible sur sensea dans l'outil DataReader

In [ ]:
files_sensea = glob.glob("./data/clean/bougival/acoustic/sensea/*.csv")
df_sensea_input = pd.concat([pd.read_csv(f) for f in files_sensea])
print("Données brut pour sensea")
df_sensea_input

In [ ]:
df_sensea_output = datareader.from_dataframe(df=df_sensea_input) # Lecture du fichier à l'aide du dataReader
print("Fichier de sortie pour sensea")
df_sensea_output